# SFO Survey Machine Learning Example

Each year, San Francisco Airport (SFO) conducts a customer satisfaction survey to find out what they are doing well and where they can improve. The survey gauges satisfaction with SFO facilities, services, and amenities. SFO compares results to previous surveys to discover elements of the guest experience that are not satisfactory.

The 2013 SFO Survey Results consists of customer responses to survey questions and an overall satisfaction rating with the airport. We investigated whether we could use machine learning to predict a customer's overall response given their responses to the individual questions. That in and of itself is not very useful because the customer has already provided an overall rating as well as individual ratings for various aspects of the airport such as parking, food quality and restroom cleanliness. However, we didn't stop at prediction instead we asked the question: 

***What factors drove the customer to give the overall rating?***

Here is an outline of our data flow:
* *Load data*: Load the data as a DataFrame
* *Understand the data*: Compute statistics and create visualizations to get a better understanding of the data to see if we can use basic statistics to answer the question above.
* *Create Model* On the training dataset:
  * *Evaluate the model*: Now look at the test dataset.  Compare the initial model with the tuned model to see the benefit of tuning parameters.
  * *Feature Importance*: Determine the importance of each of the individual ratings in determining the overall rating by the customer

This dataset is available as a public dataset from https://www.sf.gov/departments/city-administrator/datasf.

##Load the Data

In [0]:
survey = spark.read.csv("dbfs:/databricks-datasets/sfo_customer_survey/2013_SFO_Customer_Survey.csv", header="true", inferSchema="true")

Use ```display()``` to confirm the data has been loaded.

In [0]:
display(survey)

## Understand the Data

Gain a basic understanding of the data by looking at the schema.

In [0]:
survey.printSchema()

As you can see above there are many questions in the survey including what airline the customer flew on, where do they live, etc. For the purposes of answering the above, focus on the Q7A, Q7B, Q7C .. Q7O questions since they directly related to customer satisfaction, which is what you want to measure. If you drill down on those variables you get the following:

| Column Name | Data Type | Description |    
|----|----|----|  
| Q7A_ART | INTEGER | Artwork and Exhibitions |    
| Q7B_FOOD | INTEGER | Restaurants |  
| Q7C_SHOPS | INTEGER | Retail shops and concessions |  
| Q7D_SIGNS | INTEGER | Signs and Directions inside SFO |  
| Q7E_WALK | INTEGER | Escalators / elevators / moving walkways |  
| Q7F_SCREENS | INTEGER | Information on screens and monitors |  
| Q7G_INFOARR | INTEGER | Information booth near arrivals area |   
| Q7H_INFODEP | INTEGER | Information booth near departure areas |  
| Q7I_WIFI | INTEGER | Airport WiFi |  
| Q7J_ROAD | INTEGER | Signs and directions on SFO airport roadways |  
| Q7K_PARK | INTEGER | Airport parking facilities |  
| Q7L_AIRTRAIN | INTEGER | AirTrain |  
| Q7M_LTPARK | INTEGER | Long term parking lot shuttle |  
| Q7N_RENTAL | INTEGER | Airport rental car center |  
| Q7O_WHOLE | INTEGER | SFO Airport as a whole |  

The possible values for the above are:

0 = no answer, 1 = Unacceptable, 2 = Below Average, 3 = Average, 4 = Good, 5 = Outstanding, 6 = Not visited or not applicable

Select only the fields you are interested in.

In [0]:
dataset = survey.select("Q7A_ART", "Q7B_FOOD", "Q7C_SHOPS", "Q7D_SIGNS", "Q7E_WALK", "Q7F_SCREENS", "Q7G_INFOARR", "Q7H_INFODEP", "Q7I_WIFI", "Q7J_ROAD", "Q7K_PARK", "Q7L_AIRTRAIN", "Q7M_LTPARK", "Q7N_RENTAL", "Q7O_WHOLE")

Let's get some basic statistics such as looking at the average of each column.  

Let's start with the overall rating.


In [0]:
from pyspark.sql.functions import *
dataset.selectExpr('avg(Q7O_WHOLE) Q7O_WHOLE').take(1)

The overall rating is only 3.87, so slightly above average. Let's get the averages of the constituent ratings:

In [0]:
avgs = dataset.selectExpr('avg(Q7A_ART) Q7A_ART', 'avg(Q7B_FOOD) Q7B_FOOD', 'avg(Q7C_SHOPS) Q7C_SHOPS', 'avg(Q7D_SIGNS) Q7D_SIGNS', 'avg(Q7E_WALK) Q7E_WALK', 'avg(Q7F_SCREENS) Q7F_SCREENS', 'avg(Q7G_INFOARR) Q7G_INFOARR', 'avg(Q7H_INFODEP) Q7H_INFODEP', 'avg(Q7I_WIFI) Q7I_WIFI', 'avg(Q7J_ROAD) Q7J_ROAD', 'avg(Q7K_PARK) Q7K_PARK', 'avg(Q7L_AIRTRAIN) Q7L_AIRTRAIN', 'avg(Q7M_LTPARK) Q7M_LTPARK', 'avg(Q7N_RENTAL) Q7N_RENTAL')
display(avgs)

Looking at the bar chart below - the overall average rating of the airport is actually lower than all of the individual ratings. This appears clearer using the bar chart visualization below.

In [0]:
display(dataset)

So basic statistics can't seem to answer the question: ***What factors drove the customer to give the overall rating?*** 

So let's try to use a predictive algorithm to see if these individual ratings can be used to predict an overall rating.

## Create Model

Here use a decision tree algorithm to create a predictive model.

First need to treat responses of 0 = No Answer and 6 = Not Visited or Not Applicable as missing values. One of the ways you can do this is a technique called *mean impute* which is when we use the mean of the column as a replacement for the missing value. You can use a `replace` function to set all values of 0 or 6 to the average rating of 3. You also need a `label` column of type `double` so do that as well.

In [0]:
training = dataset.withColumn("label", dataset['Q7O_WHOLE']*1.0).na.replace(0,3).replace(6,3)

In [0]:
display(training)

Define the machine learning pipeline.

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

inputCols = ['Q7A_ART', 'Q7B_FOOD', 'Q7C_SHOPS', 'Q7D_SIGNS', 'Q7E_WALK', 'Q7F_SCREENS', 'Q7G_INFOARR', 'Q7H_INFODEP', 'Q7I_WIFI', 'Q7J_ROAD', 'Q7K_PARK', 'Q7L_AIRTRAIN', 'Q7M_LTPARK', 'Q7N_RENTAL']
va = VectorAssembler(inputCols=inputCols,outputCol="features")
dt = DecisionTreeRegressor(labelCol="label", featuresCol="features", maxDepth=4)
evaluator = RegressionEvaluator(metricName = "rmse", labelCol="label")
grid = ParamGridBuilder().addGrid(dt.maxDepth, [3, 5, 7, 10]).build()
cv = CrossValidator(estimator=dt, estimatorParamMaps=grid, evaluator=evaluator, numFolds = 10)
pipeline = Pipeline(stages=[va, dt])

Now call ```fit``` to build the model.

In [0]:
model = pipeline.fit(training)

Now call `display` to view the tree.

In [0]:
display(model.stages[-1])

Call `transform` to view the predictions.

In [0]:
predictions = model.transform(training)
display(predictions)

##Evaluate the model

Validate the model using root mean squared error (RMSE).

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator()

evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})

Save the model.

In [0]:
import uuid
model_save_path = f"/tmp/sfo_survey_model/{str(uuid.uuid4())}"
model.write().overwrite().save(model_save_path)

##Feature Importance

Now let's look at the feature importances. The variable is shown below.

In [0]:
model.stages[1].featureImportances

Feature importance is a measure of information gain. It is scaled from 0.0 to 1.0. As an example, feature 1 in the example above is rated as 0.1245 or 12.45% of the total importance for all the features.

Let's map the features to their proper names to make them easier to read.

In [0]:
featureImportance = model.stages[1].featureImportances.toArray()
featureNames = map(lambda s: s.name, dataset.schema.fields)
featureImportanceMap = zip(featureImportance, featureNames)

Let's convert this to a DataFrame so you can view it and save it so other users can rely on this information.

In [0]:
importancesDf = spark.createDataFrame(
    sc.parallelize(featureImportanceMap).map(lambda r: [r[1], float(r[0])])
)

In [0]:
importancesDf = importancesDf.withColumnRenamed("_1", "Feature").withColumnRenamed("_2", "Importance")

In [0]:
display(importancesDf.orderBy(desc("Importance")))

As you can see below, the 3 most important features are:

1. Signs 
2. Screens
3. Food

This is useful information for the airport management. It means that people want to first know where they are going. Second, they check the airport screens and monitors so they can find their gate and be on time for their flight.  Third, they like to have good quality food.

This is especially interesting considering that taking the average of these feature variables told us nothing about the importance of the variables in determining the overall rating by the survey responder.

These 3 features combine to make up 88% of the overall rating.

In [0]:
importancesDf.orderBy(desc("Importance")).limit(3).agg(sum("Importance")).take(1)

You can also show this graphically.

In [0]:
display(importancesDf.orderBy(desc("Importance")))

In [0]:
display(importancesDf.orderBy(desc("Importance")).limit(5))

So if you run SFO, artwork and shopping are nice-to-haves but signs, monitors, and food are what keep airport customers happy!

In [0]:
# delete saved model
dbutils.fs.rm('/tmp/sfo_survey_model',True)